![HolyChurninityLogo](Trinity-overlay-overlay-overlay.jpg)

![HolyChurninityLogo2](ProfLOGO.png)

# Swan Teleco _Churn Prediction and Risk Management_

### Summary

The aims of this project are find key predictors in _Swan Teleco's_ customer churn to implement business decisions for the Customer Retention Marketing team.

Our objective aims are to:
- Find customers most at risk to churning
- Identify key data points that indicate a customers risk of churning, deducing factors to incentivise
- Quantify the churn risk for all remaining customers

In [3]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns